<a href="https://colab.research.google.com/github/ValentinaEmili/Sign_language/blob/main/DataAugmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# mount google drive on colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from tqdm import tqdm
import numpy as np
import os

In [ ]:
def main():
  num_classes = 100 # {100, 2000}
  if num_classes == 100:
    folder = "/content/drive/MyDrive/NLP/dataset/subset_100/"
  else:
    folder = "/content/drive/MyDrive/NLP/dataset/"

  training_folder = folder + "train/"
  validation_folder = folder + "val/"
  test_folder = folder + "test/"

  training_video = training_folder + "video/"
  validation_video = validation_folder + "video/"
  test_video = test_folder + "video/"

  if num_classes == 100:
    training_images = training_folder + "images_no_vis/"
    validation_images = validation_folder + "images_no_vis/"
    test_images = test_folder + "images_no_vis/"
  else:
    training_images = training_folder + "images/"
    validation_images = validation_folder + "images/"
    test_images = test_folder + "images/"

  # images with video_id < new_id belong to the original dataset (no augmentated data)
  new_id = 70380

  all_labels = [image.split("_")[0] for image in os.listdir(training_images)]
  set_labels = set(all_labels)
  # label_counts = {class : n. samples for class}
  label_counts = {label: all_labels.count(label) for label in set_labels}

  def add_gaussian_noise(seq, mean=0, std=0.01):
    return seq + np.randorm.normal(mean, std, seq.shape)

  def jitter_joints(seq, jitter_scale=0.02):
    return np.random.uniform(-jitter_scale, jitter_scale, seq.shape)

  def random_dropout(seq, drop_prob=0.1):
    mask = np.random.binomial(1, 1 - drop_prob, seq.shape)
    return seq * mask

  def data_augmentation(images, max_count, new_id):
    added_images = 0
    for image in os.listdir(images):
      word, _ = image.split("_")
      if label_counts[word] < max_count:
        added_images += 3
        np_array = np.load(os.path.join(training_images, image))

        gauss_array = add_gaussian_noise(np_array)
        gauss_image = f"{word}_{new_id}.npy"
        new_id += 1
        np.save(os.path.join(training_images, gauss_image), gauss_array)

        jitter_points_array = jitter_joints(np_array)
        jitter_image = f"{word}_{new_id}.npy"
        new_id += 1
        np.save(os.path.join(training_images, jitter_image), jitter_points_array)

        drop_frames_array = random_dropout(np_array)
        drop_frames_image = f"{word}_{new_id}.npy"
        new_id += 1
        np.save(os.path.join(training_images, drop_frames_image), drop_frames_array)

  if num_classes == 100:
    data_augmentation(training_images, 8, new_id)
  else:
    data_augmentation(training_images, 3, new_id)

if __name__ == "__main__":
  main()

WLASL100

original dataset:
- training set: 914
- validation set: 211
- test set: 189

after:
- training set: 1,289
- validation set: 211
- test set: 189

WLASL2000

original dataset:
- training set: 8,760
- validation set: 2,472
- test set: 2,059

after:
- training set: 10,950
- validation set: 2,472
- test set: 2,059